In [ ]:
# matplotlib 한글 깨짐 해결
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 이 셀 실행 후 런터임 재실행을 실시하세요

In [ ]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import explained_variance_score
from sklearn.preprocessing import OneHotEncoder
import lightgbm as lgb



import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') # 한글설정

In [ ]:
# 파이썬 지수를 정수로 표현!
pd.options.display.float_format = '{:.5f}'.format

# 코랩 그래프 크게!

%matplotlib inline

plt.rcParams['figure.figsize'] = (20, 20)

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [ ]:

# 한글설정
plt.rc('font', family='NanumBarunGothic')

# 파이썬 지수를 정수로 표현
pd.options.display.float_format = '{:.5f}'.format

# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

# 데이터 불러오기
test = pd.read_csv('/content/drive/My Drive/2020빅콘테스트 예측/data/새미프로젝트용/test.csv', encoding = 'cp949')
train = pd.read_csv('/content/drive/My Drive/2020빅콘테스트 예측/data/새미프로젝트용/train.csv', encoding = 'cp949')




In [ ]:
# train / test 나누기
temp_train = train.drop(['Unnamed: 0','방송일시', '월', '그룹코드', '상품코드', '상품명', '일', '요일', '반기', '분기', '365일', '53주차', '시간', '시간hour', '날짜', '날짜시간', '168시간', '방송끝나는시간', '마더코드', '시청률평균', '시청률중앙값', '시청률최고값', '0.016평균값', '취급액'], axis = 1, inplace = False)
temp_test = test.drop(['Unnamed: 0', '방송일시', '월', '그룹코드', '상품코드', '상품명', '일', '요일', '반기', '분기', '365일', '53주차', '시간', '시간hour', '날짜', '날짜시간', '168시간', '방송끝나는시간', '마더코드', '취급액'], axis = 1, inplace = False)
y_train = temp_train['취급액boxcox']
y_test = temp_test['취급액boxcox']

all_data = temp_train.append(temp_test)
all_data = all_data.drop(['취급액boxcox'], axis = 1, inplace = False)

all_data = all_data.loc[:, ['노출(분)', '판매단가', '휴일', '매진여부', '한달_상품빈도', '결제수단', 'sale단어', '요일프라임비율', '24시간프라임비율', '168시간프라임비율', '그룹코드별전체횟수', '그룹코드별대박횟수', '대박확률', '월_COS', '분기_COS', '분기_SIN', '168시간_COS', '168시간_SIN', '24시간_COS', '24시간_SIN', '53주차_COS', '53주차_SIN', '중분류', '기온', '강수량', '풍속', '습도', '적설', '전운량', '비눈여부', '비눈여부_평균이상', '미세먼지', '초미세먼지']]

# 원핫 인코딩
data = pd.get_dummies(data = all_data, columns = ['휴일', '매진여부', '결제수단', 'sale단어', '중분류', '비눈여부', '비눈여부_평균이상'], drop_first = True)
x_train = data.iloc[:32480, ]
x_test = data.iloc[32480:35466, ]

print('x_train 크기 : ', x_train.shape)
print('y_train : 크기 :', y_train.shape)
print('x_test 크기 : ', x_test.shape)
print('y_test : 크기 :', y_test.shape)


In [ ]:
## Grid Search전 Feature Importance 보기

temp_model1 = lgb.LGBMRegressor()
temp_model1.fit(x_train, y_train)
temp_model1.feature_importances_

df = {'feature' : pd.Series(x_train.columns), 'importance' : pd.Series(temp_model1.feature_importances_)}
importance_df = pd.DataFrame(df)
importance_df = importance_df.sort_values(by = ['importance'])
importance_df.plot.barh(y = 'importance', x = 'feature')


## > 중분류 : 변수 중요도 수치가 거의 없다. >>> 제거

In [ ]:
## 변수 중요도가 낮은 변수 제거 (1) : 중분류
temp_all_data = all_data.drop(['중분류'], axis = 1, inplace = False)

# 원핫 인코딩
data = pd.get_dummies(data = temp_all_data, columns = ['휴일', '매진여부', '결제수단', 'sale단어', '비눈여부', '비눈여부_평균이상'], drop_first = True)
x_train = data.iloc[:32480, ]
x_test = data.iloc[32480:35466, ]

print('x_train 크기 : ', x_train.shape)
print('y_train : 크기 :', y_train.shape)
print('x_test 크기 : ', x_test.shape)
print('y_test : 크기 :', y_test.shape)

# 중분류 제거후 중요도 확인
temp_model2 = lgb.LGBMRegressor()
temp_model2.fit(x_train, y_train)
temp_model2.feature_importances_
## Feature Importance
df = {'feature' : pd.Series(x_train.columns), 'importance' : pd.Series(temp_model2.feature_importances_)}
importance_df = pd.DataFrame(df)
importance_df = importance_df.sort_values(by = ['importance'])
importance_df.plot.barh(y = 'importance', x = 'feature')

In [ ]:
## 변수 중요도가 낮은 변수 제거 (2) : sale단어
temp_all_data = all_data.drop(['중분류', 'sale단어'], axis = 1, inplace = False)

# 원핫 인코딩
data = pd.get_dummies(data = temp_all_data, columns = ['휴일', '매진여부', '결제수단', '비눈여부', '비눈여부_평균이상'], drop_first = True)
x_train = data.iloc[:32480, ]
x_test = data.iloc[32480:35466, ]

print('x_train 크기 : ', x_train.shape)
print('y_train : 크기 :', y_train.shape)
print('x_test 크기 : ', x_test.shape)
print('y_test : 크기 :', y_test.shape)

# 중분류 제거후 중요도 확인
temp_model3 = lgb.LGBMRegressor()
temp_model3.fit(x_train, y_train)
temp_model3.feature_importances_
## Feature Importance
df = {'feature' : pd.Series(x_train.columns), 'importance' : pd.Series(temp_model3.feature_importances_)}
importance_df = pd.DataFrame(df)
importance_df = importance_df.sort_values(by = ['importance'])
importance_df.plot.barh(y = 'importance', x = 'feature')

In [ ]:
# MAPE 구해주는 함수

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import explained_variance_score
from sklearn.preprocessing import OneHotEncoder
import lightgbm as lgb



In [ ]:
np.random.seed(123)

kfold = KFold(n_splits = 3, random_state=1, shuffle = True)

lgb_model = lgb.LGBMRegressor()

param = {
    'objective' : ['regression'],
    'num_leaves' : [65, 70],
    'feature_fraction' : [0.83, 0.82],
    'bagging_fraction' : [0.02, 0.03],
    'reg_alpha' : [1.5, 1.4, 1.3],
    'n_estimator' : [40 ,30]

}

lgb_grid = GridSearchCV(estimator = lgb_model,
                   param_grid = param,
                   cv = kfold,
                   scoring = 'neg_mean_absolute_error', 
                   n_jobs = 1,
                   verbose = 2)

lgb_grid.fit(x_train, y_train)


In [ ]:

print(lgb_grid.best_params_)
print(lgb_grid.best_score_)


# 8
# {'bagging_fraction': 0.02, 'feature_fraction': 0.83, 'n_estimator': 40, 'num_leaves': 70, 'objective': 'regression', 'reg_alpha': 1.4}
# -19.087826905183935

# 7
# {'bagging_fraction': 0.02, 'feature_fraction': 0.83, 'n_estimator': 40, 'num_leaves': 70, 'objective': 'regression', 'reg_alpha': 1.4}    <<<< 선택
# -19.087826905183935

# 6
# {'bagging_fraction': 0.03, 'feature_fraction': 0.84, 'n_estimator': 50, 'num_leaves': 70, 'objective': 'regression', 'reg_alpha': 1.4}
# -19.087826905183935

# 5
# {'bagging_fraction': 0.04, 'feature_fraction': 0.85, 'n_estimator': 70, 'num_leaves': 65, 'objective': 'regression', 'reg_alpha': 1.3}
# -19.098680142128668

# 4
# {'bagging_fraction': 0.05, 'feature_fraction': 0.85, 'n_estimator': 75, 'num_leaves': 65, 'objective': 'regression', 'reg_alpha': 1.2}
# -19.100643854093224

# 3
# {'bagging_fraction': 0.05, 'feature_fraction': 0.8, 'n_estimator': 100, 'num_leaves': 60, 'objective': 'regression', 'reg_alpha': 1.2}
# -19.10595081383654

# 2
# {'bagging_fraction': 0.1, 'feature_fraction': 0.8, 'n_estimator': 50, 'num_leaves': 60, 'objective': 'regression', 'reg_alpha': 1.2}
# -19.10595081383654


# 1
# {'bagging_fraction': 0.2, 'feature_fraction': 0.8, 'n_estimator': 100, 'num_leaves': 60, 'objective': 'regression', 'reg_alpha': 1.2}
# -19.10595081383654


In [ ]:
## Featrure Importance
data = {'feature' : pd.Series(x_train.columns), 'importance' : pd.Series(lgb_grid.best_estimator_.feature_importances_)}
importance_df = pd.DataFrame(data)
importance_df = importance_df.sort_values(by = ['importance'])
importance_df.plot.barh(y = 'importance', x = 'feature')



In [ ]:
# 최종모델
# {'bagging_fraction': 0.02, 'feature_fraction': 0.83, 'n_estimator': 40, 'num_leaves': 70, 'objective': 'regression', 'reg_alpha': 1.4}    <<<< 선택
# -19.087826905183935
final_model = lgb.LGBMRegressor(objective = 'regression',
                                num_leaves = 70,
                                feature_fraction = 0.83,
                                bagging_fraction = 0.02,
                                reg_alpha = 1.4,
                                n_estimator = 4)
final_model.fit(x_train, y_train)
y_pred = final_model.predict(x_test)

In [ ]:
plt.rcParams['figure.figsize'] = (20, 5)

data = {
    'real' : y_test,
    'pred' : y_pred
}

df = pd.DataFrame(data)

df.plot.line(linewidth = 0.5)



In [ ]:
# MAE 구해주는 함수
def mean_absolute_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs(y_true - y_pred))


# MAPE 구해주는 함수
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
print('MAE :',mean_absolute_error(y_test,y_pred))
print('MAPE :',mean_absolute_percentage_error(y_test,y_pred))